# Data Science for Social Justice Workshop: Module 4

## Word Embeddings

In this notebook, we'll work with word embeddings using [`gensim`](https://radimrehurek.com/gensim/models/word2vec.html).

The goal of word embedding models is to learn **numerical representations** of text corpora. We already did that to a certain extent when we did topic modeling. In this case, we're going to be more explicit about how we construct that numerical representation: for each word, we're going to find a **vector** of numbers to represent it. The actual numbers themselves won't be meaningful to us as humans. However, if successful, the vectors for each term should encode information about the meaning or concept the term represents, as well as the relationship between it and other terms in the vocabulary.

Word vector models are fully **unsupervised**: they learn all of these meanings and relationships without any advance knowledge. Unsupervised learning requires the specification of a right task. We won't go into detail in this lesson, but you can roughly think of the task as predicting nearby words, given a specific word. Read [this post](https://tomvannuenen.medium.com/analyzing-reddit-communities-with-python-part-6-word-embeddings-f92bba876d60) for a deeper introduction to word embeddings.

This notebook is designed to help you:

* Use `gensim`'s [`word2vec`](https://radimrehurek.com/gensim/models/word2vec.html) method to create word vectors for a corpus;
* Use these word vectors to reflect on implicit binaries and normativities in your data;
* Visualize topic models using K-means clustering.

In [ ]:
#Uncomment and run the code in this cell if you are running this notebook on DataHub or Binder
#!pip install spacy
#!pip install gensim
#!pip install pandas
#!pip install -U scikit-learn 
#!python -m spacy download en_core_web_sm

## Data Preprocessing

As we will be considering the language biases in the next notebook, we will use the comments of the AITA subreddit this time. The thinking behind this is that this data will be derived from more people, and include more evaluative statements (after all, comments on r/amitheasshole generally evaluate the original posts).

In [ ]:
# Package imports
import os
import pandas as pd
import pickle
import sklearn

In [ ]:
#check sklearn version
sklearn.__version__

If your sklearn version < 1.0, run the following line:

In [ ]:
#!pip install -U scikit-learn 

In [ ]:
# Change directory
# We include two ../ because we want to go two levels up in the file structure
os.chdir("../../data")

In [ ]:
# Import dataset
df = pd.read_csv('aita_sub_top_sm.csv')
df.head(3)
print(df.shape)

Next, we remove comments that were removed or deleted, and additionally only take comments that are sufficiently long:

In [ ]:
# Remove comments that are [removed] or [deleted]
df = df[~df['selftext'].isin(['[removed]', '[deleted]'])].dropna(subset=['selftext'])
# Remove comments less than 15 characters long
df = df[df['selftext'].str.len() >= 15]
len(df)


Now, we'll import `spacy` and `gensim` to do some preprocessing. We have functions written here for you to help streamline the process.

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
from gensim.models.phrases import Phrases, Phraser

In [ ]:
def clean(token):
    """Helper function that specifies whether a token is:
        - punctuation
        - space
        - digit
    """
    return token.is_punct or token.is_space or token.is_digit

def line_read(df, text_col='selftext'):
    """Generator function to read in text from df and get rid of line breaks."""    
    for text in df[text_col]:
        yield text.replace('\n', '')

def preprocess(df, text_col='selftext', allowed_postags=['NOUN', 'ADJ']):
    """Preprocessing function to apply to a dataframe."""
    for parsed in nlp.pipe(line_read(df, text_col), batch_size=1000, disable=["tok2vec", "ner"]):
        # Gather lowercased, lemmatized tokens
        tokens = [token.lemma_.lower() if token.lemma_ != '-PRON-'
                  else token.lower_ 
                  for token in parsed if not clean(token)]
        # Remove specific lemmatizations, and words that are not nouns or adjectives
        tokens = [lemma
                  for lemma in tokens
                  if not lemma in ["'s",  "’s", "’"] and not lemma in allowed_postags]
        # Remove stop words
        tokens = [token for token in tokens if token not in spacy.lang.en.stop_words.STOP_WORDS]
        yield tokens

We apply the `preprocess()` function to each comment in the dataframe, producing a `docs` output. This call might take a while.

In [ ]:
docs = [line for line in preprocess(df, text_col='selftext')]

Now, we create bigrams. Bigrams consist of pairs of words that appear commonly together (e.g., "New York"). `gensim` provides some functions to detect bigrams by finding words appear often enough together that we should include them.

In [ ]:
# Create bigram model: pass docs into Phrases class
bigrams = Phrases(docs, min_count=20, threshold=300)
# Create a "frozen" bigram model using the Phraser class
bigram_phraser = Phraser(bigrams)
# Now, create bigrams 
docs_bigrams = [bigram_phraser[doc] for doc in docs]

There's nothing stopping us from going further: we can create trigrams or even $n$-grams. We'll make some trigrams and build our word2vec model on top of them. A trigram can be constructed by simply looking for bigrams in a bigrams corpus.

In [ ]:
trigrams = Phrases(bigrams[docs], min_count=20, threshold=100)  
trigram_phraser = Phraser(trigrams)
docs_trigrams = [trigram_phraser[doc] for doc in docs_bigrams]

Let's save the data to an external JSON file:

In [ ]:
import json

In [ ]:
with open('aita_com_top_lemmas.json', 'w') as write:
    json.dump(docs_trigrams, write)

In [ ]:
# Opening the same file works as follows:
with open("aita_com_top_lemmas.json") as f:
    trigrams = json.load(f)

## Constructing a Word2Vec Model

Let's create our word embeddings model. 

While last week's LDA method was focused on finding topics in a collection of documents (or in our case, submissions), word embeddings models focus on individual words, and learning vector representations of these words.

The input to the model is a text corpus split up into sentences – in word embeddings, there is no concept of "documents". The model's output is a set of "vectors" (one for each word) in N dimensions (a parameter of the model). Think of these vectors as "features", capturing latent meaning.

This model allows us to group the vectors of similar words together in vector space. We can then reduce the dimensionality to visualize the results in a way humans can understand (such as in a 2-dimensional space), or to perform linear algebra operations in order to find out to what extent words are related.

Word2Vec is one example of a word embeddings model. It learns by taking words and their contexts (e.g. sentences) into account, and can then try to predict other words. Given enough data, usage and contexts, word2vec can make accurate guesses about a word’s meaning based on its appearances. Those guesses can be used to establish a word’s association with other words (e.g. "Paris" is to "France" as “Berlin” is to “Germany”), or cluster documents and classify them by topic.

We now instantiate and train our Word2Vec model, using the parameters below.

In [ ]:
from gensim.models import Word2Vec
import multiprocessing

In [ ]:
# Count the number of cores you have at your disposal
cores = multiprocessing.cpu_count()
# Word vector dimensionality (how many features each word will be given)
n_features = 300
# Minimum word count to be taken into account
min_word_count = 10
# Number of threads to run in parallel (equal to your amount of cores)
n_workers = cores
# Context window size
window = 5
# Downsample setting for frequent words
downsampling = 1e-2
# Seed for the random number generator (to create reproducible results)
seed = 1 
# Skip-gram = 1, CBOW = 0
sg = 1
epochs = 20

model = Word2Vec(
    sentences=trigrams,
    workers=n_workers,
    vector_size=n_features,
    min_count=min_word_count,
    window=window,
    sample=downsampling,
    seed=seed,
    sg=sg)

In [ ]:
model.train(trigrams, total_examples=model.corpus_count, epochs=10)        

That was it! We have a Word Embeddings model now. Let's save it so that we don't have to train it again. Then, we can reload the embeddings so that we don't have to train it every single time:

In [ ]:
model.save('aita.emb')

In [ ]:
model = Word2Vec.load('aita.emb')

How many terms are in our vocabulary? Whenever interacting with the word vector dictionary, we use the `wv` attribute:

In [ ]:
len(model.wv)

Let's take a peek at the word vectors our model has learned. We can take a look at the individual words using the `index_to_key` attribute, and the word vectors themselves can be accessed with the `vectors` attribute:

In [ ]:
model.wv.index_to_key[0]

In [ ]:
model.wv.vectors[0]

Looking at it won't make a whole lot of sense to us! It's just a bunch of numbers. However, we can do semantic operations on these vectors, such as getting related terms.

### Word Similarity

With the information in our word embeddings model, we can try to find similarities between words that interest us (i.e. words that have a similar vector). Let's create a function that retrieves related terms to some input. We're going to use the [`most_similar()`](https://tedboy.github.io/nlps/generated/generated/gensim.models.Word2Vec.most_similar.html) function in `gensim` as part of this helper function.

In [ ]:
def get_most_similar_terms(model, token, topn=20):
    """Look up the top N most similar terms to the token."""
    for word, similarity in model.wv.most_similar(positive=[token], topn=topn):
        print(f"{word}: {round(similarity, 3)}")

In [ ]:
get_most_similar_terms(model, 'asshole')

Here are some other terms. What else interests you?

In [ ]:
get_most_similar_terms(model, 'empathy')

In [ ]:
get_most_similar_terms(model, 'relationship')

In [ ]:
get_most_similar_terms(model, 'power')

In [ ]:
get_most_similar_terms(model, 'man')

In [ ]:
get_most_similar_terms(model, 'woman')

### Word Analogies

One of the most famous usages of `word2vec` is via word analogies. For example:

`Paris : France :: Berlin : Germany`

Here, the analogy is between (Paris, France) and (Berlin, Germany), with "capital city" being the concept that connects them. We can abstract the "analogy" relationship to vector modeling. Let's pretend we're working with each of the vectors. Then, the analogy is

$$\mathbf{v}_{\text{France}} - \mathbf{v}_{\text{Paris}} \approx \mathbf{v}_{\text{Germany}} - \mathbf{v}_{\text{Berlin}}.$$

The vector difference here represents the notion of "capital city". Presumably, going from the Paris vector to the France vector (i.e., the vector difference) will be the same as going from the Berlin vector to the Germany vector, if that difference carries similar semantic meaning.

Let's test this directly. We'll do so by rewriting the above expression:

$$\mathbf{v}_{\text{France}} - \mathbf{v}_{\text{Paris}} + \mathbf{v}_{\text{Berlin}} \approx \mathbf{v}_{\text{Germany}}.$$

The core idea is that once words are represented as numerical vectors, you can do "math" with them. In `gensim`, this works with the `most_similar` function, which takes `positive` and `negative` arguments. In the above scenario, the positive terms would be Berlin and France, while the negative term is Paris. You can roughly think of this as: "What is the vector most similar to Berlin and France, but opposite Paris?"

We can't do this example in our corpus, because we don't have all these words represented. Another example we can do is perhaps t he most well known example:

`Man : King :: Woman : ?`

What does the function tell us is on the other side of the analogy? Remember, analogies are constructions by humans: they quite literally encode semantic relationships, and thus enforce norms. The fact that the word embedding learns this analogy implies that it has inherited norms practiced by humans, whether those norms are biased or not.

In [ ]:
model.wv.most_similar(positive=['woman', 'king'], negative='man')


## Clustering Word Vectors

One convenience of word embeddings is that we can "cluster" them. We can find a group of word vectors that are close to each other, and call it a related "cluster". Since we expected word vectors that are semantically similar to be close to each other in space, we might expect the clusters to be semantically meaningful. The clustering algorithm we use is called **K-means clustering**. 

K-Means clustering aims to group $N$ observations into $K$ clusters (we choose $K$) in which each observation belongs to the cluster with the nearest mean (called the "cluster center"), which serves as a prototype of the cluster.

Since our words are all represented as vectors, applying K-means is easy to do since the clustering algorithm will simply look at differences between vectors (and centers).

A package called [`scikit-learn`](https://scikit-learn.org/stable/) provides us a couple algorithms that will be useful for this section: [`KMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) and [`KDTree`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html?highlight=kdtree#sklearn.neighbors.KDTree)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree

Here is a helper function to perform the clustering:

In [ ]:
def clustering_on_wordvecs(word_vecs, n_clusters):
    """Clusters a set of word vectors and returns the center of each cluster."""
    # Initalize a k-means object and use it to extract centroids
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++')
    cluster_ids = kmeans.fit_predict(word_vecs)
    return kmeans.cluster_centers_, cluster_ids

We can access the raw word vectors with the `vectors` attribute:

In [ ]:
n_clusters = 20
centers, cluster_ids = clustering_on_wordvecs(model.wv.vectors, n_clusters)
centroid_map = dict(zip(model.wv.index_to_key, centers))

Next, we get words in each cluster that are closest to the cluster center. To do this, we initialize a data structure called a KDTree on the word vectors, and query it for the top $K$ words on each cluster center. We will use a helper function to print this into a convenient dataframe:

In [ ]:
def get_top_words(model, n_closest, centers):
    """Get the words closest to each cluster center."""
    # Create KD Tree
    tree = KDTree(model.wv.vectors)
    # Use closest points for each cluster center to query the closest points to it
    closest_points = tree.query(centers, k=n_closest)[1]
    # Query word index for each position
    closest_words = {}
    for cluster_idx, cluster in enumerate(closest_points):
        closest_words[f'Cluster {cluster_idx + 1}'] = [model.wv.index_to_key[idx] for idx in cluster]
    # Create DataFrame from dictionary
    df = pd.DataFrame(closest_words)
    return df

Let’s get the top 50 words for each cluster:

In [ ]:
get_top_words(model, 50, centers)

## Visualizing High Dimensional Spaces with $t$-SNE

The word embeddings we created are what's called a **high-dimensional representation** of the text. That is, we take a word in the corpus, and represent it using, in this case, 300 numbers. We can plot 3 numbers at a time - that's in 3 dimensions - but there's no way for humans to visualize something in a 300-dimensional space. 

So, **dimensionality reduction** is a big part of machine learning. How can we take vectors that are 300-dimensional, and visualize them in 2-dimensions, while keeping the structure between vectors the same? How can we reduce the dimensionality?

One of the most popular methods for dimensionality reduction is called $t$-SNE ($t$-Distributed Stochastic Neighbor Embedding). The details are not important, but using it in practice is a useful skill to learn (if you want to read more, [here](https://lvdmaaten.github.io/tsne/) is a good starting point). Roughly, it tries to keep the relative distances between points as closely as possible in both high-dimensional and low-dimensional space.

So, we'll use $t$-SNE to take all the word vectors, and obtain a **low dimensional representation**. We can then visualize it, which may reveal semantic and syntactic trends in the data.

A [$t$-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html?highlight=tsne#sklearn.manifold.TSNE) implementation is available via `scikit-learn`:

In [ ]:
#if you get error in the line tsne=TSNE(), you can try to run the following line:
#!pip install -U scikit-learn 

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# Create some filepaths to save our model
tsne_path = 'tsne_model'
tsne_vectors_path = 'tsne_vectors.pkl'

In [ ]:
tsne = TSNE(init='pca', learning_rate='auto')
tsne_vectors = tsne.fit_transform(model.wv.vectors)


We have our low dimensional representation. Now, let's store the 2 dimensions in a dataframe, with the word as the index:

In [ ]:
# Store the t-SNE vectors
tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(model.wv.index_to_key),
                            columns=['x', 'y'])

In [ ]:
with open(tsne_path, 'wb') as f:
    pickle.dump(tsne, f)

tsne_vectors.to_pickle(tsne_vectors_path)

Here's a convenient code block to load this data, to start from this point:

In [ ]:
with open(tsne_path, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.read_pickle(tsne_vectors_path)

We're going to visualize the 2-dimensional space using a package called `bokeh`. This package is nice for this because it allows for some degree of interactivity: we can go over each point and dynamically get information about the word denoting that vector.

In [ ]:
!pip install bokeh

In [ ]:
import bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

output_notebook()
bokeh.io.output_notebook()

In [ ]:
# Add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# Create the plot and configure the title, dimensions, and tools
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width=800,
                   plot_height=800)

# Add a hover tool to display words on roll-over
tsne_plot.add_tools(HoverTool(tooltips='@index') )

# Draw the words as circles on the plot
tsne_plot.circle('x', 'y',
                 source=plot_data,
                 color='blue',
                 line_alpha=0.2,
                 fill_alpha=0.1,
                 size=10,
                 hover_line_color='black')

# Configure visual elements of the plot
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# Engage!
show(tsne_plot)

# Reflection: The Hermeneutics of Word Embeddings

“In vector space, identities and differences change in nature. Similarity and belonging no longer rely on resemblance or a common genesis but on measures of proximity or distance, on flat loci that run as vectors through the space.” (Dourish 2018: 73-4)

As we've seen, word embeddings are essentially a set of vectors. We should reflect on this. What is vectorization? It is reducing linguistic complexity. Or rather, it produces a common space that juxtaposes and mixes complex localized realities. Anything can be turned into a vector operation, but what do we lose when doing so? 